In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import *
from scipy.ndimage import zoom

In [2]:
year = '2019'

ref_ds = gdal.Open('/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_LST_WARP.tif')
gdriver = gdal.GetDriverByName('GTiff')
output_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/tau_clear/'

for i in range(1,13,1):
    if growingSeasonChecker(i):

        # load files for the respective month in selected year
        tcwv_file =[file for file in getFilelist(
            '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/total_column_water_vapour/', '.grib'
            ) if year in file and int(file.split('_')[-1].split('.')[0]) == i][0]


        aod_file =[file for file in getFilelist(
            '/data/Aldhani/eoagritwin/et/Auxiliary/AOD_CAMS/', '.grib'
            ) if year in file and int(file.split('_')[-1].split('.')[0]) == i][0]
        
        # open both files and find timewise matching bands
        tcwv_times = []
        aod_times = []

        tcwv_ds = gdal.Open(tcwv_file)
        for b in range(tcwv_ds.RasterCount):
            band = tcwv_ds.GetRasterBand(b + 1)
            tcwv_times.append(datetime.fromtimestamp(int(band.GetMetadata()['GRIB_VALID_TIME']), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))
        
        aod_ds = gdal.Open(aod_file)
        for b in range(aod_ds.RasterCount):
            band = aod_ds.GetRasterBand(b + 1)
            aod_times.append(datetime.fromtimestamp(int(band.GetMetadata()['GRIB_VALID_TIME']), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))

        tau_list = []
        time_list = []

        for i1, t1 in enumerate(tcwv_times):
            for i2, t2 in enumerate(aod_times):
                if t1 == t2:
                    time_list.append(t1)
                    tcwv = tcwv_ds.GetRasterBand(i1 + 1).ReadAsArray()
                    aod550 = aod_ds.GetRasterBand(i2 + 1).ReadAsArray()

                    zoom_factors = (tcwv.shape[0] / aod550.shape[0], tcwv.shape[1] / aod550.shape[1])
                    aod550_rs = zoom(aod550, zoom_factors, order=1)
                    aod700 = aod550_rs * (700 / 550) ** -1.3
                    tcwv_cm = tcwv / 10.0
                    tau_clear = np.exp(-0.9 * aod700) * np.exp(-0.15 * tcwv_cm)
                    tau_list.append(tau_clear)

        # first out_ds in memory
        out_ds = gdal.GetDriverByName('MEM').Create('', tau_clear.shape[1], tau_clear.shape[0] ,len(tau_list), gdal.GDT_Float32)
        out_ds.SetProjection(tcwv_ds.GetProjection())
        out_ds.SetGeoTransform(tcwv_ds.GetGeoTransform())
        for b0, tau in enumerate(tau_list):
            out_ds.GetRasterBand(b0 + 1).WriteArray(tau)
            out_ds.GetRasterBand(b0 + 1).SetDescription(time_list[b0])

        # warp it to LST resolution

        # Get geotransform and projection from reference
        gt = ref_ds.GetGeoTransform()
        proj = ref_ds.GetProjection()

        # Calculate bounds
        xmin = gt[0]
        ymax = gt[3]
        xres = gt[1]
        yres = -gt[5]
        xmax = xmin + ref_ds.RasterXSize * xres
        ymin = ymax - ref_ds.RasterYSize * yres

        # translated_ds = gdal.Translate(
        # '',                     # output to memory
        # grib_path,              # input file
        # format='MEM',
        # bandList=[1, 3]         # select specific bands (1-based index)
        # )

        warp_options = gdal.WarpOptions(
            format='GTiff',#'MEM',
            dstSRS=proj,
            xRes=xres,
            yRes=yres,
            outputBounds=(xmin, ymin, xmax, ymax),
            resampleAlg='bilinear',
            dstNodata=-9999,
            # targetAlignedPixels=True
        )

        # Perform reprojection and resampling
        # warped_ds = gdal.Warp('', translated_ds, options=warp_options)
        warped_ds = gdal.Warp(f"{output_path}Tau_clear_{'_'.join(aod_file.split('.')[0].split('_')[-2:])}.tif", out_ds, options=warp_options)

/data/Aldhani/users/potzschf/conda/envs/cds_era5/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Warning 1: GRIB: Don't understand the parameter table, since center 98-0 used
parameter table version 210 instead of 3 (international exchange).
Using default for now (which might lead to erroneous interpretation), but please email arthur.taylor@noaa.gov
about adding this table to his 'degrib1.c' and 'grib1tab.c' files.
Warning 1: GRIB: Parameter 207 is > 127, so it falls in the local use section of
the parameter table (and is undefined on the international table.
Using default for now(which might lead to erroneous interpretation), but please email arthur.taylor@noaa.gov
about adding this table to his 'degrib1.c' and 'grib1tab.c' files.
Warning 1: GRIB: Don't understand the parameter table, since center 98-0 used
parameter 

In [ ]:
# # export it adiabaticsharpened

# bandCount = warped_ds.RasterCount
# bandList = [rc+1 for rc in range(bandCount)]

# out_ds = gdriver.Create(f"{output_path}Tau_clear_{'_'.join(aod_file.split('.')[0].split('_')[-2:])}.tif",
#                         warped_ds.RasterXSize, warped_ds.RasterYSize,
#                         bandCount, gdal.GDT_Float32)

# out_ds.SetGeoTransform(warped_ds.GetGeoTransform())
# out_ds.SetProjection(warped_ds.GetProjection())


# # load sharpener datasets 
# dem_ds = checkPath('/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_LST_WARP.tif')
# dem = dem_ds.GetRasterBand(1).ReadAsArray()
# geopot_ds = checkPath('/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/geopotential/geopotential_low_res.tif')
# geopot = geopot_ds.GetRasterBand(1).ReadAsArray()

# for i in bandList:
#     band = warped_ds.GetRasterBand(i)
#     bandname = band.GetDescription()
#     in_arr = band.ReadAsArray()
#     out_ds.GetRasterBand(i).WriteArray(applyAdiabaticDEMsharpener(era5_temp=in_arr,
#                                                                     dem=dem,
#                                                                     geopot=geopot,
#                                                                     rate=1,
#                                                                     bheight=100))
#     out_ds.GetRasterBand(i).SetNoDataValue(-9999)
#     out_ds.GetRasterBand(i).SetDescription(bandname)


# del out_ds